In [1]:
import os
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
#os.getenv('OPENAI_API_KEY')

In [4]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
client = OpenAI(max_tokens=50)

### Individual Chain

In [5]:
prompt = PromptTemplate.from_template("Suggest me a good company name making {product}")
chain = LLMChain(llm=client, prompt=prompt, verbose=True)
chain.invoke("footballs")

/home/lal/dev/practice/langchain-basics-using-pinecone-chromadb-openai/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Suggest me a good company name making footballs

> Finished chain.


{'product': 'footballs',
 'text': '\n\n1. "Strike Force Footballs"\n2. "Goal Master Footballs"\n3. "ProKick Football Co."\n4. "Precision Play Footballs"\n5. "Gridiron Gear Co."\n6. "Endzone Elite Footballs"\n'}

In [6]:
chain.invoke("orange juice")



> Entering new LLMChain chain...
Prompt after formatting:
Suggest me a good company name making orange juice

> Finished chain.


{'product': 'orange juice',
 'text': '\n\n1. Zesty Squeeze \n2. Orange Oasis \n3. Citrus Crush \n4. Freshly Squeezed Co. \n5. Tangy Twist \n6. Orange Delight Co. \n7. Sunshine Sips'}

### Multiple Chains using Simple Sequential Chaining

In [7]:
from langchain.chains import SimpleSequentialChain
client = OpenAI(max_tokens=100)

In [8]:
company_name_prompt = PromptTemplate.from_template("Suggest me a company name for making {product}")
company_name_chain = LLMChain(llm=client, prompt=company_name_prompt)

In [9]:
company_strategies_prompt = PromptTemplate.from_template("Suggest few strategies for the this company {company}")
company_strategies_chain = LLMChain(llm=client, prompt=company_strategies_prompt)

In [10]:
chain = SimpleSequentialChain(chains=[company_name_chain, company_strategies_chain])

In [11]:
print(chain.invoke("headphones"))

{'input': 'headphones', 'output': '\n\n1. Expand product line: One strategy for the company could be to expand its product line to include a wider range of audio equipment, such as speakers, amplifiers, and recording equipment. This can help attract a broader customer base and increase sales.\n\n2. Partner with music artists and influencers: Collaborating with popular music artists and influencers can help the company gain more exposure and credibility. These partnerships can also lead to co-branded products and promotions, which can attract new customers.\n\n3. Offer'}


### Multiple Chains using Simple Sequential Chaining

In [12]:
from langchain.chains import SequentialChain

In [13]:
company_name_prompt = PromptTemplate.from_template("Suggest me a company name for making {product}")
company_name_chain = LLMChain(llm=client, prompt=company_name_prompt, output_key="company")

In [14]:
company_strategies_prompt = PromptTemplate.from_template("Suggest few strategies for the this company {company}")
company_strategies_chain = LLMChain(llm=client, prompt=company_strategies_prompt, output_key="strategies")

In [15]:
chain = SequentialChain(
    chains=[company_name_chain, company_strategies_chain],
    input_variables=["product"],
    output_variables=["company", "strategies"]
)

In [16]:
print(json.dumps(chain.invoke("orange juice"), indent=4))

{
    "product": "orange juice",
    "company": "\n\n\"Zestful Squeezes\"",
    "strategies": " \n\n1. Focus on product quality and uniqueness: Zestful Squeezes should prioritize creating high quality and unique products to stand out in the market. This could include experimenting with different flavors, ingredients, and packaging to make their products stand out from competitors.\n\n2. Collaborate with local farmers and suppliers: In order to ensure the freshness and quality of their products, Zestful Squeezes can partner with local farmers and suppliers to source their ingredients. This will not only support the"
}
